In [ ]:
import networkx as nx
import heapq
from collections import defaultdict, deque
import math

class ChristofidesSolver: 
    def __init__(self, filename):

        distance_matrix = []
        euc_coordinates = []
        explicit = True

        with open(filename, 'r') as f:
            lines = f.readlines()
            read_distances = False  

            for line in lines: 
                line = line.strip()

                if line.startswith('EOF'):
                    break
                elif line.startswith('EDGE_WEIGHT_TYPE'):
                    if 'EXPLICIT' in line:
                        explicit = True
                    else:
                        explicit = False
                elif line.startswith('EDGE_WEIGHT_SECTION') or line.startswith('NODE_COORD_SECTION'):
                    read_distances = True
                elif explicit and read_distances:
                    elements = line.split()
                    distance_matrix.append([int(e) for e in elements])
                elif explicit == False and read_distances: 
                    elements = line.split()
                    x = int(elements[1])
                    y = int(elements[2])
                    euc_coordinates.append((x, y))

        if explicit:
            self.distance_matrix = distance_matrix
            self.n = len(distance_matrix)
            self.explicit = True
        else:
            self.coordinates = euc_coordinates
            self.n = len(euc_coordinates)
            self.explicit = False

    def get_cost(self, i, j):
        if self.explicit:
            return self.distance_matrix[i][j]    
        else:
            x1, y1 = self.coordinates[i]
            x2, y2 = self.coordinates[j]
            return math.ceil(((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5)

    def prim_mst(self):

        self.mst_edges = []  # Stores (u, v, weight) edges in MST
        total_weight = 0
        visited = [False] * self.n
        min_heap = [(0, 0, -1)]  # (weight, vertex, parent)

        while len(self.mst_edges) < self.n - 1:
            weight, u, parent = heapq.heappop(min_heap)

            if visited[u]:
                continue
            
            visited[u] = True
            if parent != -1:
                self.mst_edges.append((parent, u, weight))
                total_weight += weight

            for v in range(self.n):
                if not visited[v] and self.get_cost(u, v) > 0:
                    heapq.heappush(min_heap, (self.get_cost(u, v), v, u))

    def find_odd_degree_vertices(self):
        degree = [0] * self.n

        for u, v, _ in self.mst_edges:
            degree[u] += 1
            degree[v] += 1

        self.odd_vertices = []
        for v in range(self.n):
            if degree[v] % 2 == 1:
                self.odd_vertices.append(v) 

    def minimum_weight_perfect_matching(self):
        """Find the Minimum Weight Perfect Matching (MWPM) for odd-degree vertices."""
        G = nx.Graph()
        for i in range(len(self.odd_vertices)):
            for j in range(i + 1, len(self.odd_vertices)):
                u, v = self.odd_vertices[i], self.odd_vertices[j]
                weight = self.get_cost(u, v)
                G.add_edge(u, v, weight=weight)
        mwpm = nx.algorithms.matching.min_weight_matching(G, maxcardinality=True, weight="weight")

        self.matching_edges = []
        for u, v in mwpm:
            self.matching_edges.append((u, v, self.get_cost(u, v)))

    def find_euler_circuit(self):

        self.mst_mwpm_graph = self.mst_edges.copy()  # Make a copy of mst_graph
        self.mst_mwpm_graph.extend(self.matching_edges)  # Append matching_edges to mst_graph

        # Initialize the graph with the adjacency list
        adj = defaultdict(deque)
    
        # Building the graph
        for u, v, _ in self.mst_mwpm_graph:
            adj[u].append(v)
            adj[v].append(u)

        # Check if all vertices have an even degree
        for node in adj:
            if len(adj[node]) % 2 != 0:
                print(f"Vertex {node} has an odd degree, so no Euler circuit exists.")
                return None

        # Non-recursive Hierholzer's algorithm to find Eulerian circuit
        self.euler_circuit = []
        # Start from any vertex that has edges (pick the first one)
        start_node = next(iter(adj))
        current_path = deque([start_node])
    
        while current_path:
            u = current_path[-1]

            # If there are still neighbors (edges to follow)
            if adj[u]:
                # Get the next vertex to visit
                v = adj[u].popleft()  # Get a neighbor (remove the edge)
                adj[v].remove(u)  # Remove the reverse edge
                current_path.append(v)  # Move to the next vertex
            else:
                # If there are no more neighbors, add the vertex to the circuit
                self.euler_circuit.append(current_path.pop())
    
        # Since we processed the circuit in reverse order, we reverse it to get the correct order
        self.euler_circuit.reverse()

    def christofides(self):

        self.prim_mst()

        self.find_odd_degree_vertices()

        self.minimum_weight_perfect_matching()

        self.find_euler_circuit()

        unique_tour = []
        cost = 0
        seen = set()
 
        for i, vertex in enumerate(self.euler_circuit):
            if vertex not in seen:
                unique_tour.append(vertex)
                seen.add(vertex)

                # If not the first vertex, calculate the cost from the last added vertex
                if len(unique_tour) > 1:
                    prev_vertex = unique_tour[-2]
                    cost += self.get_cost(prev_vertex, vertex)

        # To return to the start vertex (completing the cycle)
        cost += self.get_cost(unique_tour[-1], unique_tour[0])

        return cost, unique_tour

In [18]:
# filename = "../tsplib_converted_instances/01_small/gr17_converted.tsp"
filename = "../tsplib_instances_project/02_medium/eil76.tsp"
# filename = "../tsplib_converted_instances/02_medium/eil76_converted.tsp"

# Solve using 2-Opt 
christofides_solver = ChristofidesSolver(filename)
ch_cost, ch_path = christofides_solver.christofides()

print("\nChristofides-Solver - Minimum Cost:", ch_path)
print("\nPath: ", ch_cost)


Christofides-Solver - Minimum Cost: [0, 72, 61, 27, 73, 29, 1, 47, 28, 44, 26, 12, 53, 56, 14, 4, 36, 19, 69, 59, 70, 68, 35, 46, 20, 60, 51, 33, 66, 25, 11, 39, 16, 50, 5, 67, 74, 3, 75, 45, 7, 18, 34, 6, 52, 13, 58, 65, 10, 64, 37, 9, 30, 57, 71, 38, 8, 31, 49, 17, 54, 24, 43, 2, 23, 48, 15, 62, 22, 55, 32, 21, 63, 41, 40, 42]

Path:  662
